In [4]:
# Mounting colab on drive
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
# Enter the folder name
FOLDER_NAME = '/content/drive/My Drive/RutgersTask2/'

Mounted at /content/drive


In [5]:
import sys,re,json,math
from scipy import spatial
import pandas as pd

In [6]:
input_hashtag = "stopasianhate";
hashtag_re = re.compile("#\w+ ");
pattern=re.compile("[^\w#]");
threshold_words = 5; # a word should co-occur at least $threshold_words times with a hashtag
threshold_hashtags = 3; # a hashtag should co-occur at least $threshold_hashtags times with a hashtag
alpha = 0.3;
beta = 0.7; # weights for merging words and hashtags

def getList(dict1,all_things,type1):
	list1 = [];
	if(type1=='hashtag'):
		weight = math.sqrt(beta);
	else:
		weight = math.sqrt(alpha);
	for thing in all_things.keys():
		if(thing in dict1):
			list1.append(dict1[thing]*weight);
		else:
			list1.append(0.0);
	return list1;

dict_stopwords = {};
data = pd.read_csv(FOLDER_NAME+"stopasianhate.csv",sep="\t");
descriptions = list(data["Description"])

dict_hashtags = {};
dict_words = {};
all_hashtags = {}; # contains all hashtags (used in computing cosine)
all_words = {}; # contains all words (used in computing cosine)

for line in descriptions:
	try:
		line = line.strip();
	except:
		continue;
	tweet = line;
	hashtags = re.findall(hashtag_re,tweet);
	if(len(hashtags)>0):
		for hashtag in hashtags:
			hashtag = hashtag.strip();
			if(len(hashtag)<3):
				continue;
			tweet = pattern.sub(" ",tweet);
			for word in tweet.split(" "):
				word = word.strip();
				if(len(word)<3):
					continue;
				if(word in dict_stopwords):
					continue;
				if(word==hashtag.replace("#","")):
					continue;
				if(word.find("#")!=-1):
					all_hashtags[word] = 1;
					if(hashtag in dict_hashtags): # other hashtags in the tweet
						tmp = dict_hashtags[hashtag];
						if(word in tmp):
							tmp[word] += 1;
						else:
							tmp[word] = 1;
						dict_hashtags[hashtag] = tmp;
					else:
						tmp = {};
						tmp[word] = 1;
						dict_hashtags[hashtag] = tmp;
				else:
					all_words[word] = 1;
					if(hashtag in dict_words): # other words in the tweet
						tmp = dict_words[hashtag];
						if(word in tmp):
							tmp[word] += 1;
						else:
							tmp[word] = 1;
						dict_words[hashtag] = tmp;
					else:
						tmp = {};
						tmp[word] = 1;
						dict_words[hashtag] = tmp;

print(len(dict_words.keys()),file=sys.stderr)
print(len(dict_hashtags.keys()),file=sys.stderr)

# filter words/hashtags and normalize
for keys in dict_words.keys():
	max1 = 0;
	for keys1 in dict_words[keys].keys():
		if(dict_words[keys][keys1]<threshold_words):
			continue;
		if(dict_words[keys][keys1]>max1):
			max1 = dict_words[keys][keys1];

	tmp = {};
	for keys1 in dict_words[keys].keys():
		if(dict_words[keys][keys1]<threshold_words):
			continue;
#		dict_words[keys][keys1] = dict_words[keys][keys1]*1.0/max1;
		tmp[keys1] = dict_words[keys][keys1]*1.0/max1;
	dict_words[keys] = tmp;

for keys in dict_hashtags.keys():
	max1 = 0;
	for keys1 in dict_hashtags[keys].keys():
		if(dict_hashtags[keys][keys1]<threshold_hashtags):
			continue;
		if(dict_hashtags[keys][keys1]>max1):
			max1 = dict_hashtags[keys][keys1];

	tmp = {};
	for keys1 in dict_hashtags[keys].keys():
		if(dict_hashtags[keys][keys1]<threshold_hashtags):
			continue;
#		dict_words[keys][keys1] = dict_words[keys][keys1]*1.0/max1;
		tmp[keys1] = dict_hashtags[keys][keys1]*1.0/max1;
	dict_hashtags[keys] = tmp;

# compute similarity

dict_hashtag1 = dict_hashtags["#" + input_hashtag];
dict_word1 = dict_words["#" + input_hashtag];

list_hashtag1 = getList(dict_hashtag1,all_hashtags,'hashtag');
list_word1 = getList(dict_word1,all_words,'word');

dict_cosine = {};
for hashtag in dict_hashtags.keys():
	print(hashtag,file=sys.stderr)
	dict_hashtag2 = dict_hashtags[hashtag];
	if(hashtag in dict_words):
		dict_word2 = dict_words[hashtag];
	else:
		dict_word2 = {};
	list_hashtag2 = getList(dict_hashtag2,all_hashtags,'hashtag');
	list_word2 = getList(dict_word2,all_words,'word');
	cosine_similarity = 1-spatial.distance.cosine(list_hashtag1+list_word1,list_hashtag2+list_word2);
	dict_cosine[hashtag] = cosine_similarity;

for keys in dict_cosine.keys():
	print(keys + "\t" + str(dict_cosine[keys]));

Streaming output truncated to the last 5000 lines.
#newyorkbuildings
#streetlifenyc
#뉴욕시티
#mitchmcconnell
#americansystemontrial
#igotsomethintosay
#misfits
#melroseavenue
#rocknroll
#nikonusa
#isolatedeyesimage
#vibrant
#streetbeauty
#CreateChange
#SuicideAwareness
#LGBTSuicide
#DallasLGBT
#TexasLGBT
#DallasCommunity
#WordsMatter
#supportaapirestaurants
#cashapp
#referral
#savemoney
#tofuhouse
#justbtwnus
#CashBack
#Btwn
#BtwnApp
#RewardsApp
#WordOfMouth
#FoodInfluencer
#LifestyleInfluencer
#bestrestaurants
#Uniteforasianpain
#mercury
#lovehate
#RecipesAgainstRacism
#recipebook
#racismandparenting
#racismconversation
#raisingdaughters
#raisingteenagers
#parentingconversations
#growwithmanatee
#CelebritiesDotID
#InfoCelebTerkini
#partnersagainstcrime
#LatinxYoungDems
#koreancinema
#koreanfilm
#wedrawthelines
#Citizens
#Redistricting
#Commission
#state
#assembly
#SD
#BOE
#fairrepresentation
#homesellingtips
#homevalue
#sellinghomes
#fsbo
#atlrealestate
#powerhouse
#journeytowellnes

Streaming output truncated to the last 5000 lines.
#newyorkbuildings	nan
#streetlifenyc	nan
#뉴욕시티	nan
#mitchmcconnell	nan
#americansystemontrial	nan
#igotsomethintosay	nan
#misfits	nan
#melroseavenue	nan
#rocknroll	0.5408758491719396
#nikonusa	nan
#isolatedeyesimage	nan
#vibrant	nan
#streetbeauty	nan
#CreateChange	0.3873521069814252
#SuicideAwareness	nan
#LGBTSuicide	nan
#DallasLGBT	nan
#TexasLGBT	nan
#DallasCommunity	nan
#WordsMatter	nan
#supportaapirestaurants	0.37567192977719244
#cashapp	0.31175936837915397
#referral	0.31175936837915397
#savemoney	0.31175936837915397
#tofuhouse	0.27124441551504463
#justbtwnus	0.32842598467243755
#CashBack	0.32842598467243755
#Btwn	0.32842598467243755
#BtwnApp	0.32842598467243755
#RewardsApp	0.32842598467243755
#WordOfMouth	0.32842598467243755
#FoodInfluencer	0.32842598467243755
#LifestyleInfluencer	0.32842598467243755
#bestrestaurants	0.32842598467243755
#Uniteforasianpain	0.14356926884063725
#mercury	0.14356926884063725
#lovehate	0.14356926884

In [ ]:
apiToken = "PhlAcz7kVbFCmXbXMuYZFsfSbNi38uYzLSFHrspB"
